In [1]:
import json
from tqdm import tqdm

In [2]:
with open('data/events.jsonl') as f:
    events = [json.loads(line) for line in f]

In [24]:
import json
import hashlib

def escape_content(content):
    # The following characters in the content field must be escaped as shown, and all other characters must be included verbatim:
    # A line break (0x0A), use \n
    # A double quote (0x22), use \"
    # A backslash (0x5C), use \\
    # A carriage return (0x0D), use \r
    # A tab character (0x09), use \t
    # A backspace, (0x08), use \b
    # A form feed, (0x0C), use \f
    return content

def generate_event_id(pubkey, created_at, kind, tags, content):
    """Generate an event ID based on the given parameters."""
    # Escape the content
    escaped_content = escape_content(content)
    
    # Create the event structure
    event = [
        0,
        pubkey.lower(),  # Ensure the public key is in lowercase hex
        created_at,
        kind,
        tags,
        escaped_content
    ]
    
    # Serialize the event to a JSON string with no extra whitespace or line breaks
    json_string = json.dumps(event, separators=(',', ':'), ensure_ascii=False)
    
    # Hash the JSON string with SHA-256 to get the event ID
    event_id = hashlib.sha256(json_string.encode('utf-8')).hexdigest()
    
    return event_id


In [ ]:
to_resolve = []
for event in tqdm(events):
    event_id = generate_event_id(
        event['pubkey'],
        event['created_at'],
        event['kind'],
        event['tags'],
        event['content']
    ) 
    if event_id != event['id']:
        to_resolve.append(event)

In [25]:
c = 0
for event in to_resolve:
    event_id = generate_event_id(
        event['pubkey'],
        event['created_at'],
        event['kind'],
        event['tags'],
        event['content']
    ) 
    if event_id != event['id']:
        c += 1
c

28085

In [ ]:
print(event['content'].split('\n'))

['#StandWithUkraine️', '🇺🇦🇬🇧https://twitter.com/DefenceHQ/status/1674780819565727751\\n\\nOriginal', 'tweet', ':', 'https://twitter.com/ukraine_world/status/1674809838394540032\\n\\nhttps://fed.celp.de/system/media_attachments/files/110/634/280/459/465/562/original/7f751339cc434a5e.jpg\\n\\nhttps://fed.celp.de/system/media_attachments/files/110/634/280/775/749/817/original/f015782bea989248.jpg\\n\\nhttps://fed.celp.de/system/media_attachments/files/110/634/280/882/615/493/original/e0546a1c279f68a6.jpg\\n\\nhttps://fed.celp.de/system/media_attachments/files/110/634/281/143/585/053/original/22640422cd7ec735.jpg']


In [28]:
event = to_resolve[10]
event_id = generate_event_id(
    event['pubkey'],
    event['created_at'],
    event['kind'],
    event['tags'],
    event['content']
)
event_id, event['id'], event

('453373219e08f413acc136643783b89385d1a7472840d08402bf9d4ddbf728d4',
 '527fd27a1a721f480456741e4a0e94b0a0f87a74a447d10eee0bd8e8e6c83053',
 {'id': '527fd27a1a721f480456741e4a0e94b0a0f87a74a447d10eee0bd8e8e6c83053',
  'created_at': 1688145174,
  'kind': 1,
  'pubkey': 'e4197b705edc5c8c05c8bd175999e5815c0bbd6d1dc7803fce4f9e6254e41364',
  'sig': 'dec7ba1813b70e6b73e353aafa3ee5912cb7f0b5154db7aa499a6e57edf163db0366c99a78753a8ac00a691a18422e9ca77a4cb8da9b04ce8ccd04b835a77a98',
  'content': '#StandWithUkraine️ 🇺🇦🇬🇧https://twitter.com/DefenceHQ/status/1674780819565727751\\n\\nOriginal tweet : https://twitter.com/ukraine_world/status/1674809838394540032\\n\\nhttps://fed.celp.de/system/media_attachments/files/110/634/280/459/465/562/original/7f751339cc434a5e.jpg\\n\\nhttps://fed.celp.de/system/media_attachments/files/110/634/280/775/749/817/original/f015782bea989248.jpg\\n\\nhttps://fed.celp.de/system/media_attachments/files/110/634/280/882/615/493/original/e0546a1c279f68a6.jpg\\n\\nhttps://fed.

In [ ]:
import json
import hashlib
import bech32
import secp256k1

def npub_to_hex(npub: str) -> str:
    if isinstance(npub, str) == False: raise ValueError("Invalid npub")
    if npub.startswith('npub'): raise ValueError("Invalid npub")
    hrp, data = bech32.bech32_decode(npub)
    decoded_bytes = bech32.convertbits(data, 5, 8, False)
    return bytes(decoded_bytes).hex()

def sign_event_id(event_id: str, private_key_hex: str) -> str:
    private_key = secp256k1.PrivateKey(bytes.fromhex(private_key_hex))
    sig = private_key.schnorr_sign(bytes.fromhex(event_id), bip340tag=None, raw=True)
    return sig.hex()

def calc_event_id(
    public_key: str,
    created_at: int,
    kind_number: int,
    tags: list,
    content: str,
) -> str:
    data = [0, public_key, created_at, kind_number, tags, content]
    data_str = json.dumps(data, separators=(",", ":"), ensure_ascii=False)
    return hashlib.sha256(data_str.encode("UTF-8")).hexdigest()

def verify_signature(event_id: str, pubkey: str, sig: str) -> bool:
    try:
        pub_key = secp256k1.PublicKey(bytes.fromhex("02" + pubkey), True)
        result = pub_key.schnorr_verify(bytes.fromhex(event_id), bytes.fromhex(sig), None, raw=True)
        if result:
            return True
        else:
            return False
    except (ValueError, TypeError) as e:
        return False

In [27]:
event

{'id': 'e0c4411dd0ec710c3f980c0eecbeb7f6575204487e896d8717391c4bddb745bd',
 'created_at': 1684451725,
 'kind': 0,
 'pubkey': '7556987aa829381934ac6e8d9b71a4da304507f05a7ad60e447bd48218fd6faf',
 'sig': '9a6f6428cdc155052fffb71c71cc0955047998ea5b485b851c13da54663c25b43cd395d8f9dc8f8cce238f86b7b1c955ba62b0f73c598d1879a5b5de4dbc3ff9',
 'content': '{\\"display_name\\":\\"限时领券优惠\\",\\"website\\":\\"https://55uu.wang\\",\\"name\\":\\"京东拼多多抖音淘宝优惠券\\", \\"about\\":\\"京东,拼多多,抖音,淘宝,限时领券优惠\\", \\"picture\\":\\"https://tftc.io/content/images/2023/01/Screen-Shot-2023-01-30-at-11.11.27-PM.png\\",\\"nip05\\":\\"npub1w4tfs74g9yupjd9vd6xekudymgcy2plstfadvrjy002gyx8ad7hsj782lv@nostr-vip.top\\"}',
 'tags': []}

In [23]:
calc_event_id(
    events[0]['pubkey'],
    events[0]['created_at'],
    events[0]['kind'],
    events[0]['tags'],
    events[0]['content']
)

'4edb2b128c19c9be680247914e0877606f656bef324519bdc378d5680b85798e'